# 데이터 전처리
## #01. 패키지 설치

In [18]:
from pandas import read_csv, DataFrame, merge, concat
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb

## #02. 데이터 구축

### 1. 데이터 파일 열기

In [4]:
df2018 = read_csv('./data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv', encoding='euc-kr')
df2019 = read_csv('./data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='euc-kr')
df2020 = read_csv('./data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2020.csv', encoding='euc-kr')
df2021 = read_csv('./data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2021.csv', encoding='euc-kr')
df상권영역 = read_csv('./data/서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='euc-kr')
df건축물 = read_csv('./data/서울특별시 건축물대장 법정동 코드정보.csv', encoding='euc-kr')

### 2. 상권영역 데이터 구축

#### 상권 영역 데이터 확인

서울시에만 해당하는 데이터

In [5]:
df상권영역.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN
3,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN
4,201810,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,NaN


#### 법정동 코드정보

전국에 대한 데이터

In [6]:
df건축물.head()

,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일
0,42750,31224,0,강원도,영월군,산솔면 화원리,NaN,20211102,99991231
1,42750,31226,0,강원도,영월군,산솔면 석항리,NaN,20211102,99991231
2,42750,31221,0,강원도,영월군,산솔면 녹전리,NaN,20211102,99991231
3,42750,31223,312,강원도,영월군,산솔면 이목리,산솔면,20211102,99991231
4,42750,31225,312,강원도,영월군,산솔면 연상리,산솔면,20211102,99991231


In [12]:
df건축물['시군구코드'].value_counts()

시군구코드
47780    970
47740    921
47800    875
47810    835
43710    818
        ... 
27145      1
28265      1
25000      1
45118      1
22235      1
Name: count, Length: 500, dtype: int64

> `47780` 이라는 코드를 갖는 지역이 `970`곳이 됨을 확인할 수 있다. 즉, 코드값의 중복이 있다.

#### 시군구명, 시군구코드를 추출하여 중복데이터 제거

In [13]:
df시군구 = df건축물.filter(['시군구명', '시군구코드'])
df시군구 = df시군구.drop_duplicates()
df시군구.dropna(inplace=True)
df시군구

,시군구명,시군구코드
0,영월군,42750
14,유성구,30200
23,평택시,41220
30,용인시 수지구,41465
32,용인시 처인구,41461
...,...,...
82870,대천시,44170
83192,온양시,44190
86860,동구 안심출장소,22145
88397,동해출장소,42105


#### 법정동 코드 정보를 상권영역에 결합

In [14]:
df상권영역2 = merge(df상권영역, df건축물.filter(['시군구명','시군구코드']), left_on='시군구_코드', right_on='시군구코드', how='inner')
df상권영역2

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,시군구명,시군구코드
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
1,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
2,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
3,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
4,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
...,...,...,...,...,...,...,...,...,...,...,...,...
145707,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350
145708,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350
145709,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350
145710,201810,A,골목상권,1000381,동일로242길,205175,463916,11350,11350630,NaN,노원구,11350


#### 구에 대한 정보 확인

 얼마나 많은 구가 있는가?


In [15]:
df상권영역2['시군구명'].value_counts()

시군구명
종로구     14553
강남구     13915
영등포구    12284
성북구     11868
중구      11825
마포구      7739
성동구      7728
관악구      7568
동대문구     7268
용산구      5858
강서구      5313
구로구      4897
서초구      4760
서대문구     3936
강동구      3480
동작구      3250
은평구      3111
송파구      2925
양천구      2646
도봉구      2508
중랑구      2304
광진구      2040
강북구      2028
노원구      1170
금천구       738
Name: count, dtype: int64

#### 파생변수 생성

행정구는 값의 종류가 너무 많아서 더미변수화 하기가 어려울 것으로 보고 도심과 비도심으로 구분하는 파생변수를 생성
> 출처 : 서울2030플랜

In [16]:
도심 = ['종로구','중구','영등포구','서초구','강남구']

df상권영역2['도심'] = np.isin(df상권영역2['시군구명'], 도심)

df상권영역2.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,시군구명,시군구코드,도심
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
1,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
2,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
3,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
4,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False


#### 상권영역 데이터 저장

In [17]:
df상권영역2.to_excel('상권영역.xlsx', index=False)

### 3. 코로나 이전 매출 데이터

#### 단순데이터 결합

In [19]:
코로나이전df = concat([df2018, df2019])
코로나이전df.reset_index(drop=True, inplace=True)
코로나이전df

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2018,1,D,발달상권,1001046,서울 관악구 신림역_4,CS200036,고시원,83495157,281,100,0,0,67,0,33,0,0,0,0,33,0,33,33,0,100,0,0,0,33,33,0,33,83495157,0,0,75925926,0,7569231,0,0,0,0,37037037,0,7569231,38888889,0,83495157,0,0,0,37037037,38888889,0,7569231,281,0,0,186,0,95,0,0,0,0,93,0,95,93,0,281,0,0,0,93,93,0,95,0
1,2018,1,D,발달상권,1001025,서울 관악구 서울대입구역_1,CS200012,법무사사무소,6571318,116,100,0,66,0,34,0,0,0,0,0,0,34,66,0,0,66,34,0,34,66,0,0,0,6571318,0,4638577,0,1932741,0,0,0,0,0,0,1932741,4638577,0,0,2705837,3865481,0,773096,5798222,0,0,0,116,0,77,0,39,0,0,0,0,0,0,39,77,0,0,77,39,0,39,77,0,0,0,10
2,2018,1,A,골목상권,1000289,인촌로17가길,CS300021,문구,6044732,23,100,0,50,23,0,27,0,0,0,0,0,0,73,27,0,100,0,0,0,0,50,23,27,6044732,0,431767,4613085,0,999880,0,0,0,0,0,0,1431647,4613085,0,6044732,0,0,0,0,431767,999880,4613085,23,0,8,7,0,8,0,0,0,0,0,0,15,8,0,23,0,0,0,0,8,7,8,4
3,2018,1,A,골목상권,1000515,남부순환로70길,CS200030,피부관리실,11863957,40,100,0,0,0,43,57,0,0,0,0,0,25,57,18,0,0,100,0,0,0,25,75,0,11863957,0,0,0,4731978,7131979,0,0,0,0,0,4491978,7131979,240000,0,0,9463957,0,0,0,4491978,4971979,0,40,0,0,0,18,22,0,0,0,0,0,13,22,5,0,0,36,0,0,0,13,23,0,1
4,2018,1,A,골목상권,1000902,논현로63길,CS200029,네일숍,1766586,19,85,15,22,10,15,38,0,15,0,0,0,32,0,53,15,0,100,0,0,75,0,0,25,973793,792793,68650,483871,160183,261089,0,792793,0,0,0,712702,0,893701,160183,0,798844,0,0,730195,0,0,68650,16,3,4,2,3,7,0,3,0,0,0,6,0,10,3,0,15,0,0,11,0,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298239,2019,4,R,전통시장,1001469,양재시장,CS300011,일반의류,14282341,48,100,0,0,0,0,0,100,0,0,0,0,0,100,0,0,0,100,0,0,100,0,0,0,14282341,0,0,0,0,0,14282341,0,0,0,0,0,14282341,0,0,0,14282341,0,0,14282341,0,0,0,48,0,0,0,0,0,48,0,0,0,0,0,48,0,0,0,48,0,0,48,0,0,0,2
298240,2019,4,A,골목상권,1000181,망우로21길,CS100004,양식음식점,562291801,23923,55,45,8,8,8,14,18,35,11,0,0,21,7,67,5,35,65,1,23,43,13,16,4,355844273,206447528,71466001,65492851,66106572,77642229,75136620,99780650,106666878,0,0,186501020,62108268,304768715,8913798,291493403,228594333,7059099,131322960,141027943,128211368,79584958,32881410,15841,8082,3055,3108,3056,3406,3216,3967,4115,0,0,8038,2830,12827,228,12935,9718,469,6166,6432,4792,3474,1319,2
298241,2019,4,A,골목상권,1000441,세무서2길,CS300002,편의점,518371562,81848,71,29,13,15,13,15,15,15,14,15,15,11,12,26,22,70,30,3,38,19,18,16,7,369830431,148541131,68036624,76789960,67104321,77848775,80050751,81696035,66845096,80087632,74142691,55416984,59126545,142299026,107298684,357176340,161195222,9578982,154616997,101179340,113014273,101660876,38321094,58223,23625,10656,12062,10989,12168,12348,12122,11503,12334,12243,8888,9749,20980,17654,57240,24608,2813,30887,15345,14338,12869,5596,2
298242,2019,4,D,발달상권,1001044,양재역_3,CS200001,일반교습학원,15509601,56,100,0,37,33,13,7,11,0,0,0,0,0,73,20,7,54,46,0,0,17,83,0,0,15509601,0,7134416,2171344,4032497,620384,1

#### 골목상권 데이터만 추출

In [20]:
코로나이전df2 = 코로나이전df.query("상권_구분_코드_명 == '골목상권'")
코로나이전df2.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2018,1,A,골목상권,1000289,인촌로17가길,CS300021,문구,6044732,23,100,0,50,23,0,27,0,0,0,0,0,0,73,27,0,100,0,0,0,0,50,23,27,6044732,0,431767,4613085,0,999880,0,0,0,0,0,0,1431647,4613085,0,6044732,0,0,0,0,431767,999880,4613085,23,0,8,7,0,8,0,0,0,0,0,0,15,8,0,23,0,0,0,0,8,7,8,4
3,2018,1,A,골목상권,1000515,남부순환로70길,CS200030,피부관리실,11863957,40,100,0,0,0,43,57,0,0,0,0,0,25,57,18,0,0,100,0,0,0,25,75,0,11863957,0,0,0,4731978,7131979,0,0,0,0,0,4491978,7131979,240000,0,0,9463957,0,0,0,4491978,4971979,0,40,0,0,0,18,22,0,0,0,0,0,13,22,5,0,0,36,0,0,0,13,23,0,1
4,2018,1,A,골목상권,1000902,논현로63길,CS200029,네일숍,1766586,19,85,15,22,10,15,38,0,15,0,0,0,32,0,53,15,0,100,0,0,75,0,0,25,973793,792793,68650,483871,160183,261089,0,792793,0,0,0,712702,0,893701,160183,0,798844,0,0,730195,0,0,68650,16,3,4,2,3,7,0,3,0,0,0,6,0,10,3,0,15,0,0,11,0,0,5,2
5,2018,1,A,골목상권,1000407,연서로33길,CS200007,치과의원,140020113,2150,83,17,20,19,17,13,14,17,1,0,14,35,28,22,0,48,52,2,13,15,19,21,31,118081987,21938126,27870670,23643460,28061059,20582204,17924594,21887532,50594,0,32893153,51898162,28689052,26539746,0,64347602,75672511,2624519,18455405,10901884,22232839,34072474,51732992,1767,383,428,407,351,283,298,373,10,0,309,755,602,484,0,1033,1117,32,280,324,402,458,654,1
6,2018,1,A,골목상권,1000407,연서로33길,CS200008,한의원,145038784,6360,85,15,23,16,13,19,14,15,0,0,8,31,24,36,0,42,58,0,6,15,20,31,28,122468973,22569811,43795401,19120772,10201149,26598564,22753087,22569811,0,0,9818934,35375916,44467228,55065568,311138,43816057,101222727,90899,14974922,13487669,41936148,50800432,23748714,5380,980,1431,1002,834,1180,933,980,0,0,538,1978,1548,2276,20,2712,3648,10,391,896,1326,1930,1807,1


#### 코로나 이전과 이후의 조사 범위를 동일하게 설정

코로나19 발생시점인 2020년 1월을 기준으로 2018년 1,2,3,4분기, 2019년 1,2,3,4분기 데이터와 2020년 1,2,3,4분기, 2021년 1,2분기 데이터만 수집된 상태.

그러므로 코로나 이전 데이터에서 2018년 1,2분기를 분석범위에서 제외해야 코로나 전후에 대한 데이터분포가 균형을 이루게 된다.

In [24]:
제외할index = 코로나이전df2.query("(기준_분기_코드==2018 and (기준_분기_코드==1 or 기준_분기_코드==2))").index
코로나이전df3 = 코로나이전df2.drop(제외할index)
코로나이전df3.reset_index(drop=True, inplace=True)
코로나이전df3.to_excel('코로나이전_매출데이터.xlsx', index=False)
코로나이전df3

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2018,1,A,골목상권,1000289,인촌로17가길,CS300021,문구,6044732,23,100,0,50,23,0,27,0,0,0,0,0,0,73,27,0,100,0,0,0,0,50,23,27,6044732,0,431767,4613085,0,999880,0,0,0,0,0,0,1431647,4613085,0,6044732,0,0,0,0,431767,999880,4613085,23,0,8,7,0,8,0,0,0,0,0,0,15,8,0,23,0,0,0,0,8,7,8,4
1,2018,1,A,골목상권,1000515,남부순환로70길,CS200030,피부관리실,11863957,40,100,0,0,0,43,57,0,0,0,0,0,25,57,18,0,0,100,0,0,0,25,75,0,11863957,0,0,0,4731978,7131979,0,0,0,0,0,4491978,7131979,240000,0,0,9463957,0,0,0,4491978,4971979,0,40,0,0,0,18,22,0,0,0,0,0,13,22,5,0,0,36,0,0,0,13,23,0,1
2,2018,1,A,골목상권,1000902,논현로63길,CS200029,네일숍,1766586,19,85,15,22,10,15,38,0,15,0,0,0,32,0,53,15,0,100,0,0,75,0,0,25,973793,792793,68650,483871,160183,261089,0,792793,0,0,0,712702,0,893701,160183,0,798844,0,0,730195,0,0,68650,16,3,4,2,3,7,0,3,0,0,0,6,0,10,3,0,15,0,0,11,0,0,5,2
3,2018,1,A,골목상권,1000407,연서로33길,CS200007,치과의원,140020113,2150,83,17,20,19,17,13,14,17,1,0,14,35,28,22,0,48,52,2,13,15,19,21,31,118081987,21938126,27870670,23643460,28061059,20582204,17924594,21887532,50594,0,32893153,51898162,28689052,26539746,0,64347602,75672511,2624519,18455405,10901884,22232839,34072474,51732992,1767,383,428,407,351,283,298,373,10,0,309,755,602,484,0,1033,1117,32,280,324,402,458,654,1
4,2018,1,A,골목상권,1000407,연서로33길,CS200008,한의원,145038784,6360,85,15,23,16,13,19,14,15,0,0,8,31,24,36,0,42,58,0,6,15,20,31,28,122468973,22569811,43795401,19120772,10201149,26598564,22753087,22569811,0,0,9818934,35375916,44467228,55065568,311138,43816057,101222727,90899,14974922,13487669,41936148,50800432,23748714,5380,980,1431,1002,834,1180,933,980,0,0,538,1978,1548,2276,20,2712,3648,10,391,896,1326,1930,1807,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192482,2019,4,A,골목상권,1000440,세검정로1길,CS300014,신발,21965900,887,61,39,10,17,12,7,14,25,14,0,5,28,41,27,0,41,59,0,2,8,15,24,52,13153121,8812779,2302268,3264692,1849363,698228,5038570,6189703,2623076,0,943552,6944545,8133422,5944381,0,7849542,14116358,0,580772,1909067,2226783,4833591,12415687,550,337,95,171,95,57,132,225,112,0,38,265,358,226,0,366,521,0,20,79,113,211,464,3
192483,2019,4,A,골목상권,1000910,도산대로81길,CS300022,화장품,1724533532,1824,67,33,9,14,15,16,12,21,12,0,1,17,36,30,17,47,53,1,22,38,17,13,9,1132784451,591749081,193407802,200419844,218293836,276634976,244027993,325022716,266726365,0,15016718,346672956,700605385,554991357,107247116,710353177,942972814,5214748,240435852,627174832,424639513,258231861,97629184,1257,567,151,257,277,337,235,365,202,0,11,290,623,518,382,782,896,17,377,630,293,218,144,7
192484,2019,4,A,골목상권,1000181,망우로21길,CS100004,양식음식점,562291801,23923,55,45,8,8,8,14,18,35,11,0,0,21,7,67,5,35,65,1,23,43,13,16,4,355844273,206447528,71466001,65492851,66106572,77642229,75136620,99780650,106666878,0,0,186501020,62108268,304768715,8913798,291493403,228594333,7059099,131322

### 4. 코로나 이후 매출데이터

In [23]:
코로나이후df = concat([df2020, df2021])
코로나이후df.reset_index(drop=True, inplace=True)

코로나이후df2 = 코로나이후df.query("상권_구분_코드_명 == '골목상권'")
코로나이후df2.to_excel('코로나이후_매출데이터.xlsx', index=False)
코로나이후df2

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2020,1,A,골목상권,1000022,창신2길,CS200001,일반교습학원,22367209.0,82,100,0,0,71,29,0,0,0,0,0,0,0,0,100,0,71,29,0,0,0,100,0,0,22367209.0,0.0,0,13450049.0,8917160,0,0,0,0,0,0,0.0,0.0,22367209.0,0,12909615.0,9457594.0,0,0,0,22367209.0,0,0,82,0,0,54,28,0,0,0,0,0,0,0,0,82,0,54,28,0,0,0,82,0,0,2
4,2020,1,A,골목상권,1000001,계동길,CS100001,한식음식점,431442455.0,18548,79,21,13,15,16,17,18,11,10,0,2,47,13,36,2,49,51,2,31,31,16,13,7,347666954.0,83775501.0,57836408,62676591.0,76100528,74583641,76469786,44469818,39305683,0,2948631,199264316.0,50651452.0,152694784.0,25883272,191019483.0,152066866.0,2544277,69650930,91286746,72418118.0,65658966,41527310,15115,3433,2750,2882,3268,3057,3158,1795,1638,0,639,9704,3342,4518,345,8752,7776,611,3828,4356,3428,2614,1690,19
5,2020,3,A,골목상권,1000127,광나루로12길,CS200016,당구장,424034.0,17,100,0,0,0,0,47,53,0,0,0,0,0,0,47,53,100,0,0,0,100,0,0,0,424034.0,0.0,0,0.0,0,324463,99571,0,0,0,0,0.0,0.0,99571.0,324463,424034.0,0.0,0,0,424034,0.0,0,0,17,0,0,0,0,8,9,0,0,0,0,0,0,8,9,17,0,0,0,17,0,0,0,1
8,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161.0,1413,78,22,12,17,19,15,15,9,14,0,0,54,18,28,0,47,53,12,29,19,28,7,6,19300912.0,6466249.0,3122759,4094989.0,5150660,3326962,3605542,2475494,3990755,0,0,14401640.0,4027810.0,7337711.0,0,10101382.0,12469443.0,1485670,7127377,4988043,5928017.0,1514728,1526990,1099,314,176,225,262,213,223,122,192,0,0,763,251,399,0,592,694,145,368,240,368,96,69,1
9,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866.0,1272,83,17,14,19,16,18,17,17,0,0,0,47,11,23,19,55,45,0,12,22,24,28,14,46837834.0,8647032.0,6794759,10280121.0,9847923,9122449,10792582,8647032,0,0,0,18210940.0,4118222.0,14049497.0,19106207,27143276.0,19725428.0,0,4898312,10530729,11958404.0,14016411,5464850,1055,217,173,233,211,222,216,217,0,0,0,599,142,290,241,622,508,0,134,245,275,314,162,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197054,2021,1,A,골목상권,1000001,계동길,CS100005,제과점,165688274.0,18512,76,24,14,16,18,14,14,13,12,0,24,28,18,27,4,37,45,1,11,19,18,16,17,125817232.0,39871042.0,23646093,26415916.0,28922188,22966291,23866744,20898892,18972150,0,39177374,45781959.0,29726354.0,44862354.0,6140233,60546425.0,73700130.0,1175614,17667578,31323681,29874639.0,26707544,27497498,14265,4247,2921,2826,3293,2487,2738,2141,2106,0,5043,4515,3320,4958,676,7523,9187,208,2471,4577,3857,2990,2609,4
197055,2021,1,A,골목상권,1000001,계동길,CS100004,양식음식점,199448496.0,4442,54,46,8,10,12,10,14,25,21,0,0,31,19,48,2,40,43,0,28,25,9,15,6,107935451.0,91513045.0,16261937,20220422.0,23670669,20735461,27046962,49828030,41685015,0,0,62577748.0,38029880.0,94762211.0,4078657,79945162.0,85649352.0,682175,55225642,49380427,18079254.0,29516708,12710309,2427,2015,370,498,519,466,574,1129,886,0,0,1413,881,2082,66,1778,2175,31,1505,1155,439,566,256,3
197056,2021,1,A,골목상권,1000001,계동길,CS100